In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Import Data from CSV
df_test = pd.read_csv('../input/titanic/test.csv')
df_train = pd.read_csv('../input/titanic/train.csv')

#Display Data Test
df_test.head()

In [2]:
#Display Data Train
df_train.head()

In [3]:
#Inspect Data Train
df_train.dtypes

Clean Data

In [4]:
#Clean Data

    #Count Rows
index = df_train.index
number_of_rows = len(index)
print(number_of_rows)

In [5]:
    # Count total NaN at each column in a DataFrame
print(" \nCount total NaN at each column in a DataFrame : \n\n",
      df_train.isnull().sum())

In [6]:
    #Drop 'Cabin' to many NaN's, Drop 'Ticket', 'Name' as not relevant
df_train = df_train.drop(columns=['Cabin', 'Name', 'Ticket'])

    #Fill missing values in 'Age'
df_train['Age'] = df_train['Age'].interpolate()

    #Fill mising Values in 'Embarked'
df_train['Embarked'] = df_train['Embarked'].interpolate()

    # Count total NaN at each column in a DataFrame
print(" \nCount total NaN at each column in a DataFrame : \n\n",
      df_train.isnull().sum())

In [7]:
#Get Corrolation of train data
df_train.corr()

In [8]:
#Create columns for Pclass, Sex, Embarked to allow analysis and modelling

    #Copy df_train to df_train_2
df_train_2 = df_train.copy()

    #Convert Pclass, Sex, Embarked to columns and attach to df_train_2
dummies = []
cols = ['Pclass','Sex','Embarked']
for col in cols:
    dummies.append(pd.get_dummies(df_train_2[col]))
titanic_dummies = pd.concat(dummies, axis=1)
df_train_2 = pd.concat((df_train_2,titanic_dummies),axis=1)
df_train_2 = df_train_2.drop(['Pclass','Sex','Embarked'],axis=1)
df_train_2.head()

Create ML model with sklearn

In [9]:
#Create ML

    #Import sklearn
import sklearn
from sklearn.model_selection import train_test_split


    #Create Numpy array
X = df_train_2.values
y = df_train_2['Survived'].values

    #Drop 'Survived' from X
X = np.delete(X,1,axis=1)

    #Split data set to train and test 70/30 split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=0)

    #Create Decision Tree
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=5)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

Run through four different model types to find best fit.

In [10]:
from sklearn import ensemble
clf = ensemble.RandomForestClassifier(n_estimators=100)
clf.fit (X_train, y_train)
clf.score (X_test, y_test)

In [11]:
clf = ensemble.GradientBoostingClassifier()
clf.fit (X_train, y_train)
clf.score (X_test, y_test)

In [12]:
clf = ensemble.GradientBoostingClassifier(n_estimators=50)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

With the best model fitted at 83% I'm pretty happy with that, so now using 'GradientBoostingClassifier' move on to import test data set

In [13]:
df_test.head()

Perfomr same data cleaning steps to the test set to match the train set.

In [14]:
    # Count total NaN at each column in a DataFrame
print(" \nCount total NaN at each column in a DataFrame : \n\n",
      df_test.isnull().sum())

In [15]:
    #Drop 'Cabin' to many NaN's, Drop 'Ticket', 'Name' as not relevant
df_test = df_test.drop(columns=['Cabin', 'Name', 'Ticket'])

    #Fill missing values in 'Age'
df_test['Age'] = df_test['Age'].interpolate()

    #Fill missing values in 'Fare'
df_test['Fare'] = df_test['Fare'].interpolate()

    # Count total NaN at each column in a DataFrame
print(" \nCount total NaN at each column in a DataFrame : \n\n",
      df_test.isnull().sum())

In [16]:
    #Convert Pclass, Sex, Embarked to columns and attach to df_test
dummies = []
cols = ['Pclass','Sex','Embarked']
    for col in cols:
        dummies.append(pd.get_dummies(df_test[col]))
titanic_dummies = pd.concat(dummies, axis=1)
df_test = pd.concat((df_test,titanic_dummies),axis=1)
df_test = df_test.drop(['Pclass','Sex','Embarked'],axis=1)

df_test.head()

Run the ML model on the test data.

In [17]:
#Run Model clf on Test Data
X_results = df_test.values
y_results = clf.predict(X_results)

Output the results to CSV.

In [18]:
#Output to CSV for Kaggle Comp
output = np.column_stack((X_results[:,0],y_results))
df_results = pd.DataFrame(output.astype('int'),columns=['PassengerID','Survived'])
df_results.to_csv('titanic_results.csv',index=False)